# Outline for DGE Analysis
1. Load package dependencies  
2. Define custom functions   
3. Read the data  
4. Define parameters  
5. Normalize counts  
6. DGE with limma  
7. DGE with DESeq2  

## 1. Load package dependencies
Base R uses a single package repository named "CRAN" mirrored across the globe, but the bioinformatics world has established a second independent package repository named "Bioconductor".   
The Jupyter system includes some high-profile packages from CRAN that are useful for data analysis, but we also need to install a few from Bioconductor. Installing the packages takes a minute, so in this block we do a check to see if we need to install them before actually doing so.  

The `data.table` package is from CRAN, while the `BiocManager` package is our interface with Bioconductor.  
After installing the packages we must explicitly load them with the `library` command.  

In [111]:
library(data.table)

if (!requireNamespace("BiocManager", quietly = TRUE)) {
    install.packages("BiocManager")
    BiocManager::install(c("limma", "edgeR", "DESeq2"))
}
library(limma)
library(edgeR)
library(DESeq2)

install.packages('statmod')
library(statmod)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)

Installing package(s) 'DESeq2'

also installing the dependencies ‘formatR’, ‘png’, ‘Biostrings’, ‘GenomeInfoDbData’, ‘zlibbioc’, ‘matrixStats’, ‘lambda.r’, ‘futile.options’, ‘KEGGREST’, ‘XML’, ‘GenomeInfoDb’, ‘XVector’, ‘MatrixGenerics’, ‘DelayedArray’, ‘futile.logger’, ‘snow’, ‘BH’, ‘AnnotationDbi’, ‘annotate’, ‘S4Vectors’, ‘IRanges’, ‘GenomicRanges’, ‘SummarizedExperiment’, ‘BiocGenerics’, ‘Biobase’, ‘BiocParallel’, ‘genefilter’, ‘geneplotter’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'backports', 'brio', 'broom', 'bslib', 'caret', 'cli',
  'conflicted', 'cpp11', 'crayon', 'credentials', 'crosstalk', 'data.table',
  'DBI', 'desc', 'devtools', 'dials', 'diffobj', 'digest', 'DT', 'dtplyr',
  'e1071', 

## 2. Define custom functions
We will use edgeR to normalize our count data as counts per million. This process is roughly 5 calls to other functions, but we can wrap them up in a single function call.  
For clarity the package namespace is referenced before each function call using the namespace selector `::`. This is useful but not necessary. If a function namespace is not specified it is (probably) from the base packages loaded when R starts.  

In [71]:
normalizeCounts <- function(counts){
  d <- edgeR::DGEList(counts = counts, group = colnames(counts))
  dNorm <- edgeR::calcNormFactors(d, method = 'TMM')
  
  lcpm <- edgeR::cpm(dNorm, log = T)
  L <- mean(dNorm$samples$lib.size) * 1e-6
  M <- median(dNorm$samples$lib.size) * 1e-6
  
  lcpm.cutoff <- log2(10/M + 2/L)
  
  r.m.l.cpm <- rowMeans(lcpm)
  
  filtered <- which(r.m.l.cpm >= lcpm.cutoff)
  return(dNorm[filtered,])
}

## 3. Read Counts Data
The counts will come in as a matrix that requires little reshaping bit may need some cleanup.  

In [6]:
list.files()

[1] "Basic R.ipynb"                               
[2] "bioinformatics-bootcamp"                     
[3] "champions-starter-materials"                 
[4] "lost+found"                                  
[5] "RSEM_expected_counts_matrix_2021-03-11.RData"
[6] "sample_counts.csv"                           
[7] "Untitled.ipynb"

In [49]:
dat <- data.table::fread('./sample_counts.csv')

Rather than start blind, we can explore the data and see what the file contains. R has a few built-in functions to explore tables -- we can view the top few rows with `head`, examine the structure with `str`, and get a summary of the fields with `summary`.  We also should confirm that this is an object of class `data.frame` so we know what methods are available to us.  

In [14]:
class(dat)

[1] "data.table" "data.frame"

In [17]:
dim(dat)

[1] 33562    13

In [27]:
head(dat)

,A_1,B_1,vehicle_1,C_1,A_2,C_2,vehicle_2,C_3,B_2,A_3,B_3,vehicle_3
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,98,198,202,219,198,183,225,182,253,97,337,192
2,231,265,192,158,301,137,179,153,262,272,211,121
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,3,0,0,0,0
5,31,1,27,19,13,21,26,17,11,22,4,7
6,1039,2410,65,1021,1667,359,369,339,895,920,1139,703


In [28]:
str(dat)

'data.frame':	30145 obs. of  12 variables:
 $ A_1      : int  98 231 0 0 31 1039 0 0 4 0 ...
 $ B_1      : int  198 265 0 0 1 2410 0 0 0 0 ...
 $ vehicle_1: int  202 192 0 0 27 65 0 0 1 0 ...
 $ C_1      : int  219 158 0 0 19 1021 0 0 25 0 ...
 $ A_2      : int  198 301 0 0 13 1667 0 2 10 0 ...
 $ C_2      : int  183 137 0 0 21 359 0 0 5 0 ...
 $ vehicle_2: int  225 179 0 0 26 369 0 0 3 0 ...
 $ C_3      : int  182 153 0 3 17 339 0 0 4 0 ...
 $ B_2      : int  253 262 0 0 11 895 0 0 2 0 ...
 $ A_3      : int  97 272 0 0 22 920 0 0 2 0 ...
 $ B_3      : int  337 211 0 0 4 1139 0 1 3 0 ...
 $ vehicle_3: int  192 121 0 0 7 703 0 0 2 0 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [29]:
summary(dat)

      A_1               B_1            vehicle_1            C_1         
 Min.   :      0   Min.   :      0   Min.   :      0   Min.   :      0  
 1st Qu.:      0   1st Qu.:      0   1st Qu.:      0   1st Qu.:      0  
 Median :      8   Median :      5   Median :      8   Median :      8  
 Mean   :   1682   Mean   :   1432   Mean   :   1465   Mean   :   1589  
 3rd Qu.:    988   3rd Qu.:    574   3rd Qu.:    751   3rd Qu.:    917  
 Max.   :1055229   Max.   :4056283   Max.   :1810834   Max.   :1057202  
      A_2               C_2           vehicle_2            C_3        
 Min.   :      0   Min.   :     0   Min.   :      0   Min.   :     0  
 1st Qu.:      0   1st Qu.:     0   1st Qu.:      0   1st Qu.:     0  
 Median :     11   Median :     7   Median :      7   Median :     7  
 Mean   :   2418   Mean   :  1221   Mean   :   1508   Mean   :  1305  
 3rd Qu.:   1292   3rd Qu.:   701   3rd Qu.:    847   3rd Qu.:   656  
 Max.   :1983211   Max.   :579935   Max.   :1127776   Max.   :9

The first column appears to be row names, which we should remove. Remove it by using a negative column index and reassigning the value to its prior name.  

In [50]:
dat <- dat[,-1]
head(dat)

Gene,A_1,B_1,vehicle_1,C_1,A_2,C_2,vehicle_2,C_3,B_2,A_3,B_3,vehicle_3
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
A1BG,98,198,202,219,198,183,225,182,253,97,337,192
A1BG-AS1,231,265,192,158,301,137,179,153,262,272,211,121
A1CF,0,0,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,3,0,0,0,0
A2M-AS1,31,1,27,19,13,21,26,17,11,22,4,7
A2ML1,1039,2410,65,1021,1667,359,369,339,895,920,1139,703


The data.table package is one of the few packages that allows us to modify objects in-place, so in general you should explicitly state that you are replacing the object with a modified version.  

For DGE analysis we need to a numeric matrix with samples in columns and genes in rows. We will store the gene symbols, trim the data to just numbers, then set the row names and drop the Gene field before coercing to matrix. 

Some of the gene symbols have been inadvertently read in as dates. Instead of dealing with this issue I'm going to remove them and ignore it, but this is worth chasing down and fixing someday.  

In [51]:
dat <- dat[!grepl("[0-9]-[A-Za-z]*", dat$Gene),]
r_names <- dat$Gene
dat <- dat[, -1]
dat_m <- as.matrix(dat)
rownames(dat_m) <- r_names
class(dat_m)
head(dat_m)

[1] "matrix" "array"

,A_1,B_1,vehicle_1,C_1,A_2,C_2,vehicle_2,C_3,B_2,A_3,B_3,vehicle_3
A1BG,98,198,202,219,198,183,225,182,253,97,337,192
A1BG-AS1,231,265,192,158,301,137,179,153,262,272,211,121
A1CF,0,0,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,3,0,0,0,0
A2M-AS1,31,1,27,19,13,21,26,17,11,22,4,7
A2ML1,1039,2410,65,1021,1667,359,369,339,895,920,1139,703


## 4. Define parameters
We can specify an *a priori* p value threshold, plus we also need to identify which samples we will compare in group A vs group B.

In [99]:
threshold <- 0.01

samples <- list(
  A = c('vehicle_1', 'vehicle_2', 'vehicle_3'),
  B = c('A_1', 'A_2', 'A_3')
  #B = c('B_1', 'B_2', 'B_3')
  #B = c('C_1', 'C_2', 'C_3')
)

## 5. Normalize counts

In [72]:
dat_norm <- normalizeCounts(dat_m)
dat_lcpm <- edgeR::cpm(dat_norm, log = T, prior.count = 1)

We also need to prepare a design matrix for the model that estimates t-statistics for sample group.

In [79]:
test_models <- data.frame()
for (i in names(samples)){
  test_models <- rbind(
    test_models, 
    data.frame(Model = samples[[i]], Group = rep(i, length(samples[[i]])))
  )
}
test_models

Model,Group
<chr>,<chr>
vehicle_1,A
vehicle_2,A
vehicle_3,A
A_1,B
A_2,B
A_3,B


The samples will be coded A, B, and NotInStudy. 

In [92]:
samplenames <- data.frame(Model = colnames(dat_lcpm))
test_design <- merge(
    samplenames, 
    test_models, 
    by = 'Model', 
    all.x = TRUE,
    sort = FALSE
)
test_design

Model,Group
<chr>,<chr>
A_1,B
vehicle_1,A
A_2,B
vehicle_2,A
A_3,B
vehicle_3,A
B_2,NA
B_1,NA
B_3,NA


In [93]:
test_design[is.na(test_design$Group), 'Group'] <- 'NotInStudy'
test_design$Group <- factor(test_design$Group, levels = unique(test_design$Group))
test_design

Model,Group
<chr>,<fct>
A_1,B
vehicle_1,A
A_2,B
vehicle_2,A
A_3,B
vehicle_3,A
B_2,NotInStudy
B_1,NotInStudy
B_3,NotInStudy


A critical detail is that our row positions in the design matrix must match the column positions in our count data, or else the results will be scrambled.

In [94]:
rownames(test_design) <- test_design$Model
test_design <- test_design[colnames(dat_lcpm),]
test_design

,Model,Group
,<chr>,<fct>
A_1,A_1,B
B_1,B_1,NotInStudy
vehicle_1,vehicle_1,A
C_1,C_1,NotInStudy
A_2,A_2,B
C_2,C_2,NotInStudy
vehicle_2,vehicle_2,A
C_3,C_3,NotInStudy
B_2,B_2,NotInStudy


In [95]:
design <- model.matrix(~0+Group, test_design)
design

,GroupB,GroupA,GroupNotInStudy
A_1,1,0,0
B_1,0,0,1
vehicle_1,0,1,0
C_1,0,0,1
A_2,1,0,0
C_2,0,0,1
vehicle_2,0,1,0
C_3,0,0,1
B_2,0,0,1
A_3,1,0,0


## 6. DGE with package limma

In [144]:
contr_matrix <- limma::makeContrasts(
  AvsB = GroupA  - GroupB, 
  levels = colnames(design)
)
fit <- limma::lmFit(dat_lcpm, design = design)
vfit <- limma::contrasts.fit(fit, contrasts = contr_matrix)
efit <- limma::eBayes(vfit, robust = T, trend = T)
lim <- limma::topTable(efit, number = Inf)

res <- limma::decideTests(fit, p.value=0.01)
summary(res)

       GroupB GroupA GroupNotInStudy
Down      494    533             787
NotSig   2116   2080            1542
Up      11751  11748           12032

In [146]:
colnames(lim) <- gsub(pattern = '\\.','',colnames(lim))
head(lim)

,logFC,AveExpr,t,PValue,adjPVal,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DDT,1.642594,5.028632,9.078759,8.305839e-07,0.00901724,5.942825
ZNF787,1.481911,5.169803,8.275110,2.237760e-06,0.00901724,5.091929
FBRSL1,1.461616,6.379390,8.241889,2.334826e-06,0.00901724,5.054929
LRFN4,2.483240,3.702051,8.214309,2.511591e-06,0.00901724,4.991140
RBM38,1.085727,5.223225,7.791460,4.203860e-06,0.01026739,4.538214
TMUB1,1.330901,4.615554,7.776291,4.289696e-06,0.01026739,4.520316


## 7. DGE with package DESeq2
Unlike limma, the DESeq2 package requires we supply it with raw counts rather than normalized counts.

In [115]:
des_dat <- DESeqDataSetFromMatrix(countData = dat_m, colData = test_design, design = design) 
des <- DESeq(des_dat)
des <- as.data.frame(results(des))
head(des)

using supplied model matrix

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A1BG,209.7286035,8.0978874,0.1871206,43.2762982,0.000000e+00,0.000000e+00
A1BG-AS1,208.8946834,7.8862533,0.2063827,38.2117949,0.000000e+00,0.000000e+00
A1CF,0.0000000,NA,NA,NA,NA,NA
A2M,0.2776903,-0.7628297,2.1919097,-0.3480206,7.278247e-01,7.712672e-01
A2M-AS1,15.6505128,3.7149824,0.4191566,8.8629938,7.789391e-19,1.217968e-18
A2ML1,940.4528834,10.3011191,0.3819061,26.9729090,3.073176e-160,5.931920e-160


In [147]:
tmp_des <- des[,c('pvalue', 'padj', 'log2FoldChange')]
tmp_des$Gene <- rownames(tmp_des)
tmp_lim <- lim[,c('PValue', 'adjPVal', 'logFC')]
colnames(tmp_lim)[1:3] <- paste0(colnames(tmp_lim)[1:3], "_limma")
colnames(tmp_des)[1:3] <- paste0(colnames(tmp_des)[1:3], "_DESeq")
tmp_lim$Gene <- rownames(tmp_lim)
comb <- merge(tmp_lim, tmp_des, on="Gene")
head(comb)

,Gene,PValue_limma,adjPVal_limma,logFC_limma,pvalue_DESeq,padj_DESeq,log2FoldChange_DESeq
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,A1BG,0.01474514,0.1902237,1.005501605,0.000000e+00,0.000000e+00,8.097887
2,A1BG-AS1,0.29018884,0.6753204,-0.450562487,0.000000e+00,0.000000e+00,7.886253
3,A2M-AS1,0.99909504,0.9998542,-0.001023591,7.789391e-19,1.217968e-18,3.714982
4,A2ML1,0.04513458,0.3235799,-1.904547068,3.073176e-160,5.931920e-160,10.301119
5,AAAS,0.79640765,0.9507145,-0.055513621,0.000000e+00,0.000000e+00,10.312942
6,AACS,0.79351349,0.9500624,0.087857330,0.000000e+00,0.000000e+00,10.892855


In [149]:
zz <- comb[c('adjPVal_limma', 'padj_DESeq')]
zz <- na.omit(zz)
cor(zz$adjPVal_limma, zz$padj_DESeq)

[1] 0.003879315

In [150]:
cor(comb$logFC_limma, comb$log2FoldChange_DESeq)

[1] -0.03104085